In [16]:
import pandas as pd
from pathlib import Path
from api_key import weather_api_key
import time
import requests

In [17]:
sightings_file = Path("usa_sightings_df_.csv")

In [18]:
sightings_df = pd.read_csv(sightings_file)
sightings_df

,SIGHTING_ID,OCCURRED_DATE,CITY,STATE,COUNTRY,SHAPE,SUMMARY,YEAR,CITYSTATE
0,1,2024-06-20,Cranston,RI,USA,Flash,My wife and I as well as our 2 children witnes...,2024.0,"Cranston, RI"
1,2,2024-06-20,Key West,FL,USA,Light,Saw unexplainable lights slightly west of Nort...,2024.0,"Key West, FL"
2,3,2024-06-20,Oklahoma City,OK,USA,Light,Erratically moving light at high altitude,2024.0,"Oklahoma City, OK"
3,4,2024-06-19,Valley Stream,NY,USA,Sphere,Sphere in the sky that glowed like an LED ligh...,2024.0,"Valley Stream, NY"
4,5,2024-06-19,Jacksonville,FL,USA,Triangle,Black triangle very low no noise,2024.0,"Jacksonville, FL"
...,...,...,...,...,...,...,...,...,...
50222,50223,2014-02-18,Chandler,IN,USA,Other,Lighted object larger than a commercial a/c ho...,2014.0,"Chandler, IN"
50223,50224,2014-02-18,Wentzville,MO,USA,Triangle,"Dark gray boomerang craft, no lights, gliding ...",2014.0,"Wentzville, MO"
50224,50225,2014-02-18,Mardela Springs,MD,USA,Light,One star-like light turned sharp and sped up t...,2014.0,"Mardela Springs, MD"
50225,50226,2014-02-18,Lame Deer,MT,USA,Unknown,Orange lights slowly from south to north.,2014.0,"Lame Deer, MT"


In [19]:
# dropping duplicates
sightings_df_copy = sightings_df.drop_duplicates(subset=['CITY'])
sightings_df_copy

,SIGHTING_ID,OCCURRED_DATE,CITY,STATE,COUNTRY,SHAPE,SUMMARY,YEAR,CITYSTATE
0,1,2024-06-20,Cranston,RI,USA,Flash,My wife and I as well as our 2 children witnes...,2024.0,"Cranston, RI"
1,2,2024-06-20,Key West,FL,USA,Light,Saw unexplainable lights slightly west of Nort...,2024.0,"Key West, FL"
2,3,2024-06-20,Oklahoma City,OK,USA,Light,Erratically moving light at high altitude,2024.0,"Oklahoma City, OK"
3,4,2024-06-19,Valley Stream,NY,USA,Sphere,Sphere in the sky that glowed like an LED ligh...,2024.0,"Valley Stream, NY"
4,5,2024-06-19,Jacksonville,FL,USA,Triangle,Black triangle very low no noise,2024.0,"Jacksonville, FL"
...,...,...,...,...,...,...,...,...,...
50193,50194,2014-02-20,Tehama,CA,USA,Chevron,Saw bright white stringy chevron shape heading...,2014.0,"Tehama, CA"
50207,50208,2014-02-19,Wirtz,VA,USA,Formation,Curved line of 6 or 7 red lights in Western sk...,2014.0,"Wirtz, VA"
50210,50211,2014-02-19,Monterey Bay,CA,USA,Fireball,Orange fire ball spotted over the Monterey Bay...,2014.0,"Monterey Bay, CA"
50225,50226,2014-02-18,Lame Deer,MT,USA,Unknown,Orange lights slowly from south to north.,2014.0,"Lame Deer, MT"


In [20]:
cities = sightings_df_copy['CITY'].to_list()
cities

['Cranston',
 'Key West',
 'Oklahoma City',
 'Valley Stream',
 'Jacksonville',
 'Lawrenceburg',
 'Leawood',
 'Washington',
 "Lee's Summit",
 'Yardley',
 'Boise',
 'Westland',
 'Del Rey',
 'Marietta',
 'Gunnison',
 'Spicer City',
 'Sacramento',
 'Bainbridge',
 'Deer Park',
 'Yakima',
 'Oyster Bay',
 'Montclair',
 'Anchor Point',
 'Battle Creek',
 'West Milford',
 'Richmond',
 'Dillingham',
 'Waterboro',
 'Palmer',
 'Troy',
 'Alamogordo',
 'Los Angeles',
 'Oldtown',
 'Boothbay',
 'Pittsburgh',
 'Wills Point',
 'Pacifica',
 'Orlando',
 'Charlotte',
 'Seattle',
 'Round Hill',
 'Winston-Salem',
 'Pawleys Island',
 'Portland',
 'Lee',
 'Cookeville',
 'Summerfield',
 'Ocala',
 'Glennville',
 'La Pine',
 'Coopersville',
 'Satellite Beach',
 'Hillsboro',
 'New York',
 'Boone',
 'Corona',
 'Willow Canyon',
 'Summerville',
 'Des Allemands',
 'Brooksville',
 'Colorado Springs',
 'Salt Lake City',
 'Orangevale',
 'North Arlington',
 'Manistique',
 'Calverton',
 'Leavenworth',
 'Tigard',
 'Cedartown

In [21]:
cities_df = pd.DataFrame(cities)
cities_df

,0
0,Cranston
1,Key West
2,Oklahoma City
3,Valley Stream
4,Jacksonville
...,...
12056,Tehama
12057,Wirtz
12058,Monterey Bay
12059,Lame Deer


In [22]:
#cities_df.to_csv("sightings_df_copy_reduced.csv")

In [23]:
base_url='https://api.openweathermap.org/data/2.5/weather?'
# url = f"{base_url}q={city}&units=imperial&appid={weather_api_key}"
# print(url)
# response = requests.get(url)
# data = response.json()
# data

## Set the API base URL
base_url = "http://api.openweathermap.org/geo/1.0/direct?"
city_data = []
# Iterate over each city in the DataFrame
for index in range(0,50226):
    if index not in sightings_df_copy.index:
        print(f"Index {index} not found in usa_cities_df. Skipping...")
        continue
    
    city_name = sightings_df_copy.loc[index, 'CITY']
    state_code = sightings_df_copy.loc[index, 'STATE']
    country_code = "US"
    
    # Construct the API request URL
    city_url = f"{base_url}q={city_name},{state_code},{country_code}&appid={weather_api_key}"
    
    print(f"Fetching data for: {city_name}, {state_code}")
    try:
        # Make the API request
        req = requests.get(city_url, verify=False)
        req.raise_for_status() 
        city_info = req.json()[0] 
        
        # Extract relevant data
        city_lat = city_info['lat']
        city_lng = city_info['lon']
        city_name = city_info['name'] 
        
        # Append the city information into city_data list
        city_data.append({"City": city_name, 
                          "Lat": city_lat, 
                          "Lng": city_lng
                          })
        
        print(f"Data retrieved successfully for: {city_name}, {state_code}")
    except (KeyError, requests.exceptions.RequestException) as e:
        # Handle KeyError (for missing keys in city_info) and RequestException (for API request failures)
        print(f"Error fetching data for: {city_name}, {state_code}")
        print(e)
        continue  # Skip to the next iteration if there's an error 
    
    except IndexError as e:
        # Handle IndexError (for cases where req.json()[0] doesn't exist)
        print(f"No data found in API response for: {city_name}, {state_code}")
        print(e)
        continue  # Skip to the next iteration if there's an error
        
    time.sleep(1)
# Print completion message
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

In [26]:
city_data

[{'City': 'Cranston', 'Lat': 41.7809588, 'Lng': -71.4371257},
 {'City': 'Key West', 'Lat': 24.5625566, 'Lng': -81.7724368},
 {'City': 'Oklahoma City', 'Lat': 35.4729886, 'Lng': -97.5170536},
 {'City': 'Village of Valley Stream', 'Lat': 40.6631362, 'Lng': -73.7056955},
 {'City': 'Jacksonville', 'Lat': 30.3321838, 'Lng': -81.655651},
 {'City': 'Lawrenceburg', 'Lat': 38.0371378, 'Lng': -84.8962551},
 {'City': 'Leawood', 'Lat': 38.966673, 'Lng': -94.6169012},
 {'City': 'Washington', 'Lat': 38.8950368, 'Lng': -77.0365427},
 {'City': "Lee's Summit", 'Lat': 38.9107156, 'Lng': -94.3821295},
 {'City': 'Yardley', 'Lat': 40.245664, 'Lng': -74.8459972},
 {'City': 'Boise', 'Lat': 43.6166163, 'Lng': -116.200886},
 {'City': 'Westland', 'Lat': 42.3238056, 'Lng': -83.4005321},
 {'City': 'Del Rey', 'Lat': 33.9896527, 'Lng': -118.4243437},
 {'City': 'Marietta', 'Lat': 33.9528472, 'Lng': -84.5496148},
 {'City': 'Gunnison', 'Lat': 38.5458246, 'Lng': -106.9253207},
 {'City': 'Spicer City', 'Lat': 35.5010719

In [27]:
city_data_df1 = pd.DataFrame(city_data)
city_data_df1

,City,Lat,Lng
0,Cranston,41.780959,-71.437126
1,Key West,24.562557,-81.772437
2,Oklahoma City,35.472989,-97.517054
3,Village of Valley Stream,40.663136,-73.705696
4,Jacksonville,30.332184,-81.655651
...,...,...,...
9389,Grand Rivers,37.003111,-88.234475
9390,Scottsville,37.798757,-78.495566
9391,Tehama,40.027102,-122.123322
9392,Wirtz,37.076529,-79.889478


In [28]:
city_data_df1.to_csv("city Data Complete 2.csv")